In [1]:
import sys
import h5py
import argparse
import os.path as path
import os
import sys
sys.path.append(path.abspath("../../../../"))
from CNN.CNN_models import *
from CNN.NeuralNet import NeuralNet
#from deeprank.learn.modelGenerator import *

def get_cnn_outputs(test_hdf5, architecture, pretrained_model, outdir, with_cuda = False):
    # try:
    #     os.makedirs(outdir)
    # except FileExistsError:
    #     pass

    model = NeuralNet(test_hdf5,
        model = architecture,
        cuda = bool(with_cuda),
        ngpu = (0,1)[with_cuda],
        outdir = outdir,
        pretrained_model=pretrained_model,
    )

    model.test()

# get_cnn_outputs(
#     test_hdf5= '/projects/0/einf2380/data/pMHCI/features_output_folder/CNN/splits_HLA_quant_allele_clusters/clustered/0/test.hdf5',
#     pretrained_model = '/projects/0/einf2380/data/pMHCI/trained_models/CNN/clustAllele_noPSSM_sumFeat_bn/CnnClassGroupConv/best_valid_model.pth.tar',
#     architecture = CnnClassGroupConv,
#     outdir = '/projects/0/einf2380/data/pMHCI/trained_models/CNN/clustAllele_noPSSM_sumFeat_bn/CnnClassGroupConv/'
#     )

In [6]:
import h5py
import pandas as pd
import sklearn.metrics as skm
import numpy as np


def sort_mlp_output(ids, mlp_outputs):
    mlp_outputs.ID = mlp_outputs.ID.astype("category")
    mlp_outputs.ID = mlp_outputs.ID.cat.set_categories(ids)
    mlp_outputs = mlp_outputs.sort_values(['ID'])
    mlp_outputs = mlp_outputs.loc[~mlp_outputs.ID.isna()]
    return mlp_outputs

c = pd.read_csv('/projects/0/einf2380/data/external/processed/I/BA_pMHCI_human_quantitative_only_eq_models_output.csv', delimiter=',')
#Shuffled
f = h5py.File('/projects/0/einf2380/data/pMHCI/trained_models/CNN/shuffled_noPSSM_sumFeat_bn/CnnClassGroupConv/test_data.hdf5')
shuff_ids = f['epoch_0000/test/mol'][:,1]
shuff_ids = [x.decode('UTF-8') for x in shuff_ids.tolist()]

mlp_shuff = c.loc[~c.mlp_shuffled_class.isna(), ['ID','mlp_shuffled_class']]
shuff_ids = [x for x in shuff_ids if x in mlp_shuff.ID.tolist()]
mlp_shuff = sort_mlp_output(shuff_ids, mlp_shuff)

cnn_shuff = [x for i, x in enumerate(f['epoch_0000/test/outputs'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in shuff_ids]
shuff_targets = [x for i, x in enumerate(f['epoch_0000/test/targets'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in shuff_ids] 

f.close()

# #Peptide
# f = h5py.File('/projects/0/einf2380/data/pMHCI/trained_models/CNN/clustPept_noPSSM_sumFeat_bn/CnnClassGroupConv/test_data.hdf5')
# pepts_ids = f['epoch_0000/test/mol'][:,1]
# pepts_ids = [x.decode('UTF-8') for x in pepts_ids.tolist()]

# mlp_pepts = c.loc[~c.mlp_peptide_clustered_class.isna(), ['ID','mlp_peptide_clustered_class']]
# pepts_ids = [x for x in pepts_ids if x in mlp_pepts.ID.tolist()]
# mlp_pepts = sort_mlp_output(pepts_ids, mlp_pepts)

# cnn_pepts = [x for i, x in enumerate(f['epoch_0000/test/outputs'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in pepts_ids]
# pepts_targets = [x for i, x in enumerate(f['epoch_0000/test/targets'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in pepts_ids] 

# f.close()

#Alleles
f = h5py.File('/projects/0/einf2380/data/pMHCI/trained_models/CNN/clustAllele_noPSSM_sumFeat_bn/CnnClassGroupConv/test_data.hdf5')
alls_ids = f['epoch_0000/test/mol'][:,1]
alls_ids = [x.decode('UTF-8') for x in alls_ids.tolist()]

mlp_alls = c.loc[~c.mlp_allele_clustered_class.isna(), ['ID','mlp_allele_clustered_class']]
alls_ids = [x for x in alls_ids if x in mlp_alls.ID.tolist()]
mlp_alls = sort_mlp_output(alls_ids, mlp_alls)

cnn_alls = [x for i, x in enumerate(f['epoch_0000/test/outputs'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in alls_ids]
alls_targets = [x for i, x in enumerate(f['epoch_0000/test/targets'][:]) if f['epoch_0000/test/mol'][:,1][i].decode('UTF-8') in alls_ids] 

f.close()



# mlp_pepts = c.loc[~c.mlp_peptide_clustered_class.isna(), ['ID','mlp_peptide_clustered_class']]
# pepts_ids = [x for x in pepts_ids if x in mlp_pepts.ID.tolist()]
# mlp_pepts = sort_mlp_output(pepts_ids, mlp_pepts)
# overlap_pepts = mlp_pepts.loc[mlp_pepts['ID'].isin(pepts_ids)]

# mlp_alls = c.loc[~c.mlp_allele_clustered_class.isna(), ['ID','mlp_allele_clustered_class']]
# alls_ids = [x for x in alls_ids if x in mlp_alls.ID.tolist()]
# mlp_alls = sort_mlp_output(alls_ids, mlp_alls)
# overlap_alls = mlp_alls.loc[mlp_alls['ID'].isin(alls_ids)]

# overlap_shuff = mlp_shuff.loc[mlp_shuff['ID'].isin(shuff_ids)]

KeyboardInterrupt: 

In [5]:
import torch

mlp_outputs = [[float(y) for y in x.split(",")] for x in mlp_shuff.mlp_shuffled_class.tolist()]
mlp_outputs = torch.softmax(torch.tensor(mlp_outputs), 1)
cnn_shuff_auc = skm.roc_auc_score(shuff_targets, np.argmax(cnn_shuff, 1))
mlp_shuff_auc = skm.roc_auc_score(shuff_targets[:len(mlp_shuff)], np.argmax(mlp_outputs, 1))

stack = torch.stack((torch.tensor(cnn_shuff[:len(mlp_outputs)]), mlp_outputs))
combined_auc = skm.roc_auc_score(shuff_targets[:len(mlp_shuff)], np.argmax(torch.mean(stack, 2), 0))

print('SHUFFLED: CNN {:.2f} MLP {:.2f} COMBINED {:.2f}'.format(cnn_shuff_auc, mlp_shuff_auc, combined_auc) )

# cnn_pepts_auc = skm.roc_auc_score(pepts_targets, np.argmax(cnn_pepts, 1))
# mlp_pepts_auc = skm.roc_auc_score(pepts_targets[:len(mlp_pepts)], np.argmax([[float(y) for y in x.split(",")] for x in mlp_pepts.mlp_peptide_clustered_class.tolist()], 1))
#
# print('PEPTIDES: CNN {:.2f} MLP {:.2f}'.format(cnn_pepts_auc, mlp_pepts_auc) )

cnn_alls_auc = skm.roc_auc_score(alls_targets, np.argmax(cnn_alls, 1))
mlp_alls_auc = skm.roc_auc_score(alls_targets[:len(mlp_alls)], np.argmax([[float(y) for y in x.split(",")] for x in mlp_alls.mlp_allele_clustered_class.tolist()], 1))

print('ALLELES: CNN {:.2f} MLP {:.2f}'.format(cnn_alls_auc, mlp_alls_auc) )


/scratch-local/dmarz/ipykernel_949548/2614814137.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/torch/csrc/utils/tensor_new.cpp:230.)
  stack = torch.stack((torch.tensor(cnn_shuff[:len(mlp_outputs)]), mlp_outputs))


SHUFFLED: CNN 0.78 MLP 0.83 COMBINED 0.49
ALLELES: CNN 0.60 MLP 0.50
